In [1]:
from selenium.webdriver import ActionChains
from common.util_function import *
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common import NoSuchElementException
from selenium.webdriver.common.by import By
from controller.db import DbManager
from datetime import datetime
from common.custom_class import NThread
import time

db_manager = DbManager("test")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
search_query = "커피"
#회사정보 입력 만들기
url = 'https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query=' + search_query

driver.get(url)

count = 1
a_click = 1
div_selector = 1
popular_blog_selector = 1
total_a_count = 0


#인기주제글 모아져있는게 없으면 인기글만 긁어오기 2개

while True:
    try:
        span_element_1 = driver.find_element(By.CSS_SELECTOR, "div > div > div:nth-of-type(2) > div > div:nth-child("+str(div_selector)+") > a:nth-child("+str(count)+") > span:nth-child(2)")
        count += 1
        total_a_count += 1
          
    except NoSuchElementException:
        div_selector += 1
        count = 1
        if div_selector == 3:
            break
        
print(total_a_count)
div_selector = 1
#블로그 2개 제한
Limit_Two = 0

while True:
    try:
        #인기 주제어 순서대로 선택
        a_click_element = driver.find_element(By.XPATH, "//*[@class='sc_new _slog_visible']/div/div/div[2]/div/div["+str(div_selector)+"]/a["+str(a_click)+"]")
        print("주소: ", a_click_element.get_attribute("href"))
        #카페 인기글은 패스 
        pass_click_cafe = driver.find_element(By.XPATH, "//*[@class='sc_new _slog_visible']/div/div/div[2]/div/div["+str(div_selector)+"]/a["+str(a_click)+"]/span[2]")
        
        if "카페" in pass_click_cafe.text:
            print("해당 요소에 '카페'라는 글이 포함되어 있습니다.")
            a_click += 1
            continue
        Limit_Two = 0    
        a_click_element.click()
        print("1")
        time.sleep(3)
        popular_blog_selector = 1
        #2개 블로그 선택해서 긁어오기
        while True:
            if Limit_Two == 2:
                driver.back()
                break
            all_blog_post_gpts = db_manager.get_all_blog_post_for_gpts()
            blog_element = driver.find_element(By.XPATH, "//*[contains(@class, 'block-mod-list')]/div["+str(popular_blog_selector)+"]/div[2]/div/div[1]/a[1]")
            time.sleep(3)
            blog_element.click()
            time.sleep(3)
            post_url = driver.current_url
            print(post_url)
            time.sleep(3)
            while True:
                try:
                    driver.switch_to.frame("mainFrame")
                    post_name = driver.find_element(By.XPATH, "//*[contains(@class, 'se-title-text')]/p/span").text
                    print(post_name)
                    time.sleep(3)
                    post_body = driver.find_element(By.CSS_SELECTOR, ".se-main-container").text
                    print(post_body)
                    driver.switch_to.default_content()
                    break
                except:
                    post_name = driver.find_element(By.XPATH, "//*[contains(@class, 'se-title-text')]/p/span").text
                    print(post_name)
                    time.sleep(3)
                    post_body = driver.find_element(By.CSS_SELECTOR, ".se-main-container").text
                    print(post_body)
                    break
            time.sleep(3)
            already_exists = 0
            if all_blog_post_gpts: 
                for post_gpt in all_blog_post_gpts:
                    if post_gpt['post_url'] == post_url or post_gpt['post_name'] == post_name or post_gpt['post_body'] == post_body:
                        already_exists += 1
                        break
                        
            if already_exists == 1:
                driver.back()
                popular_blog_selector += 2
                continue
            
            db_manager.insert_blog_post_gpt(post_name, post_body, post_url)
            driver.back()
            popular_blog_selector += 2
            Limit_Two += 1
            
            #2개를 받아오고 바로 
            
        
        a_click += 1
        total_a_count -= 1
    except NoSuchElementException:
        div_selector += 1
        a_click = 1
        if div_selector == 3:
            break


KeyboardInterrupt



In [4]:
import openai
from langchain_openai import ChatOpenAI
import os
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

openai_api_key = ''

os.environ["OPENAI_API_KEY"] = ''

chatAI = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1)

conversation = ConversationChain(llm=chatAI, verbose=True)

#글을 먼저 토큰으로 나누기

#반복문 - 키워드 당 2개 글 가져옴 - 2개당 1글 작성
#   제목 분석 post_gpt['post_name']만 제목만
#   키워드 분석 post_gpt['post_keyword']만 #키워드 긁어오기
#   내용 분석 post_gpt['post_content']만 내용만
#   위 3개 합쳐서 글 작성하기

AI_response = conversation.predict(input="내연기관 자동차를 만드는 방법에 대해서 설명해줘")
print(AI_response)
AI_response = conversation.predict(input="전기차하고 다른점은 뭐야")
print(AI_response)
AI_response = conversation.predict(input="조금전에 이야기한 것들을 한줄로 요약해줘")
print(AI_response)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 내연기관 자동차를 만드는 방법에 대해서 설명해줘
AI:



KeyboardInterrupt



In [7]:
##db_manager = DbManager("fuhafuha9")
db_manager.insert_blog_post("pgw031203", "test1", "test_name", "test_body")
all_blogs = db_manager.get_all_blogs()
all_posts = db_manager.get_all_blog_posts()

for blog in all_blogs:
    filtered_posts = [
        post for post in all_posts if post and post.get("blog_id") == blog.get("blog_id")
    ]
    
    time.sleep(1)
    url = f"https://m.blog.naver.com/{blog['blog_id']}"
    get_page(driver, url)
    time.sleep(2)
    
    current_number = 1
    
    try:
        button = driver.find_element(By.CSS_SELECTOR, '#contentslist_block > div > div > div > button:nth-of-type(2)')
        click(button)
        time.sleep(1)
        while True:
            selector = f'#contentslist_block > div > div > div:nth-of-type(2) > ul > li:nth-of-type({current_number})'
            selector_title = f'{selector} > div > a > div > strong > span > span'
            time.sleep(1)
            selector_content = f'{selector} > div > a > div > p > span > span'
            time.sleep(1)
            
            post_title = driver.find_element(By.CSS_SELECTOR, selector_title).text
            post_content = driver.find_element(By.CSS_SELECTOR, selector_content).text
            if post_content == "":
                post_content = "Post Content is None"
            time.sleep(1)
            
            latest_post = driver.find_element(By.CSS_SELECTOR, selector)
            # Extract the URL of the latest post
            latest_post_url = latest_post.find_element(By.TAG_NAME, "a").get_attribute("href")
            
            # Find the last occurrence of '/'
            last_slash_index = latest_post_url.rfind('/')
            
            # Find the index of '?' after the last '/'
            question_mark_index = latest_post_url.find('?', last_slash_index)
            time.sleep(1)
            if last_slash_index != -1 and question_mark_index != -1:
                extracted_part = latest_post_url[last_slash_index + 1:question_mark_index]
                time.sleep(1)
                continue_flag = False
                for post in filtered_posts:
                    if post["post_id"] == extracted_part:
                        current_number += 1
                        continue_flag = True
                        break
                if continue_flag:
                    continue
                print(extracted_part)
                print(post_title)
                print(post_content)
                db_manager.insert_blog_post(blog["blog_id"], extracted_part, post_title, post_content)
                print("db 넣기 완료")
                break
                    
            else:
                print("Pattern not found.")
    except:
        print('글 없음')
        continue

OperationalError: no such column: blog_id

In [5]:
db_manager = DbManager("fuhafuha9")
all_blogs = db_manager.get_all_blogs()
all_posts = db_manager.get_all_blog_posts()

now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("1")
for blog in all_blogs:
    filtered_posts = [
        post for post in all_posts
        if post and post.get("blog_id") is not None and post.get("blog_id") == blog.get("blog_id" and post.get("is_liked") != 1)
    ]
    time.sleep(1)
    if blog["neighbor_request_rmv"] == 1:
        continue
    # 현재 이웃신청이 되지 않은 블로그 중 조건을 만족하지 못한 블로그들에 대해서 Post에 좋아요와 댓글을 달도록 만듬
    if blog["neighbor_request_current"] == 0:
        if blog["like_count"] < 5 and blog["comment_count"] < 5:
            time.sleep(1)
            blog_url = f"https://m.blog.naver.com/{blog['blog_id']}"
            if filtered_posts:
                for post in filtered_posts:
                    post_url = blog_url + '/' + post['post_id']
                    get_page(driver, post_url)
                    time.sleep(1)
                    # 좋아요 버튼 확인
                    try:
                        is_liked = driver.find_element(by='xpath',
                                                    value='//*[@id="body"]/div[10]/div/div[1]/div/div/a').get_attribute(
                            'aria-pressed')  # 좋아요 버튼 상태 확인
                        if is_liked == 0:  
                            like_button = driver.find_element(By.XPATH, '//*[@id="body"]/div[10]/div/div[1]/div/div/a/span')
                            click(like_button)
                            rand_sleep(450, 550)
                            blog['like_count'] += 1
                            post['is_liked'] = 1
                    except Exception:
                        # print('공감 버튼 없음')
                        pass
                    time.sleep(1)
                    # 댓글 확인
                    try:
                        comment_button = driver.find_element(By.XPATH, '//*[@id="body"]/div[10]/div/div[2]/a[1]')
                        click(comment_button)
                        # 댓글 입력란을 찾아서 내용 입력
                        comment_input_1 = driver.find_element(By.XPATH,
                                                            '//*[@id="naverComment"]/div/div[7]/div[1]/form/fieldset/div/div/div[2]/div/label')
                        click(comment_input_1)
                        comment_input = driver.find_element(By.XPATH, '//*[@id="naverComment__write_textarea"]')
                        click(comment_input)
                        time.sleep(1)
                        comment = "안녕하세요~ 좋은 글 잘 읽었습니다!!"

                        comment_input.send_keys(comment)  # 원하는 댓글 내용으로 수정

                        # 댓글 작성 버튼을 찾아서 클릭
                        rand_sleep(450, 550)
                        comment_button = driver.find_element(By.XPATH,
                                                            '//*[@id="naverComment"]/div/div[7]/div[1]/form/fieldset/div/div/div[6]/button')
                        time.sleep(1)
                        comment_button.click()
                        blog['comment_count'] += 1
                        post['written_comment'] = comment
                        # 댓글 작성 완료 메시지 출력
                        print("댓글을 작성했습니다.")
                        db_manager.update_blog(blog)
                        db_manager.update_post(post)

                        close_current_window(driver)
                    except Exception as e:
                        print(f"An error occurred: {str(e)}")
        else:
            continue
                        # Handle the error as needed, e.g., logging or additional actions.
    else:
        if (now - blog["neighbor_request_date"]).days > 7:
            # Update neighbor_request_current to False
            blog["neighbor_request_current"] = 0
            blog["neighbor_request_rmv"] = 1
            db_manager.update_blog(blog)
        else:
            continue

BlogPostTable이 비었습니다.
1


TypeError: 'NoneType' object is not iterable

In [4]:
db_manager = DbManager("pgw031203")
all_blogs = db_manager.get_all_blogs()
all_posts = db_manager.get_all_blog_posts()

now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for blog in all_blogs:
    if blog["blog_id"] == "pgw031203":
        continue
    filtered_posts = [
        post for post in all_posts
        if post and post.get("blog_id") is not None and post.get("blog_id") == blog.get("blog_id") and post.get("is_liked") == 0
    ]
    if blog["neighbor_request_rmv"] == 1:
        continue
    blog_url = f"https://m.blog.naver.com/{blog['blog_id']}"
    if blog["neighbor_request_current"] == 0:
        if blog["like_count"] < 5 and blog["comment_count"] < 5:
            for post in filtered_posts:
                post_url = blog_url + '/' + post['post_id']
                get_page(driver, post_url)
                time.sleep(1)
                # 좋아요 버튼 확인
                try:
                    time.sleep(1)
                    is_liked = driver.find_element(By.CSS_SELECTOR,
                                                "#body > div.floating_menu > div > div.btn_like_w > div > div > a").get_attribute(
                        'aria-pressed')  # 좋아요 버튼 상태 확인
                    print(is_liked)
                    time.sleep(1)
                    if is_liked == "false":  
                        like_button = driver.find_element(By.CSS_SELECTOR, '#body > div.floating_menu > div > div.btn_like_w > div > div > a')
                        click(like_button)
                        time.sleep(1)
                        rand_sleep(450, 550)
                        blog['like_count'] += 1
                        post['is_liked'] = 1
                    else:
                        pass
                except Exception:
                    # print('공감 버튼 없음')
                    pass
                time.sleep(1)
                # 댓글 확인
                try:
                    if post['written_comment'] is not None:
                        continue
                    time.sleep(1)
                    comment_button = driver.find_element(By.CSS_SELECTOR, '#body > div.floating_menu > div > div.btn_r > a.btn_reply')
                    click(comment_button)
                    time.sleep(4)
                    # 댓글 입력란을 찾아서 내용 입력
                    comment_input_1 = driver.find_element(By.CSS_SELECTOR,
                                                        '#naverComment > div > div.u_cbox_write_wrap > div.u_cbox_write_box.u_cbox_type_logged_in > form > fieldset > div > div > div.u_cbox_write_area > div > label')
                    click(comment_input_1)
                    time.sleep(1)
                    comment_input = driver.find_element(By.CSS_SELECTOR, '#naverComment__write_textarea')
                    click(comment_input)
                    time.sleep(1)
                    comment = "안녕하세요~ 좋은 글 잘 읽었습니다!!"
                
                    comment_input.send_keys(comment)  # 원하는 댓글 내용으로 수정
                
                    # 댓글 작성 버튼을 찾아서 클릭
                    rand_sleep(450, 550)
                    time.sleep(1)
                    comment_button = driver.find_element(By.CSS_SELECTOR,
                                                        '#naverComment > div > div.u_cbox_write_wrap > div.u_cbox_write_box.u_cbox_type_logged_in > form > fieldset > div > div > div.u_cbox_upload > button')
                    time.sleep(1)
                    comment_button.click()
                    blog['comment_count'] += 1
                    post['written_comment'] = comment
                    # 댓글 작성 완료 메시지 출력
                    print("댓글을 작성했습니다.")
                    time.sleep(1)
                    db_manager.update_blog(blog)
                    db_manager.update_post(post)
                
                except Exception as e:
                    print(f"An error occurred: {str(e)}")
close_current_window(driver)

false
댓글을 작성했습니다.
false
댓글을 작성했습니다.
false
댓글을 작성했습니다.
false
댓글을 작성했습니다.
false
댓글을 작성했습니다.
false
댓글을 작성했습니다.
false
댓글을 작성했습니다.


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF695BF5E42+3538674]
	(No symbol) [0x00007FF695814C02]
	(No symbol) [0x00007FF6956C599D]
	(No symbol) [0x00007FF6956FF0F9]
	(No symbol) [0x00007FF695700631]
	GetHandleVerifier [0x00007FF695C26CAD+3738973]
	GetHandleVerifier [0x00007FF695C7C506+4089270]
	GetHandleVerifier [0x00007FF695C74823+4057299]
	GetHandleVerifier [0x00007FF695945C49+720121]
	(No symbol) [0x00007FF69582126F]
	(No symbol) [0x00007FF69581C304]
	(No symbol) [0x00007FF69581C432]
	(No symbol) [0x00007FF69580BD04]
	BaseThreadInitThunk [0x00007FF9A51E7604+20]
	RtlUserThreadStart [0x00007FF9A5CA26A1+33]


In [ ]:
introduction = "안녕하세요"
first_subtitle = "저는 첫번째 본문입니다."
second_subtitle = "저는 두번째 본문입니다."
third_subtitle = "저는 세번째 본문입니다."


main_xpath = '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[1]/div[2]/section/article/div[2]/div/div'
driver.find_element('xpath',main_xpath).click()
time.sleep(2)
ActionChains(driver).send_keys(introduction).perform()
time.sleep(random.randint(2))
ActionChains(driver).send_keys('\n\n\n'+ first_subtitle).perform()
time.sleep(random.randint(2))
ActionChains(driver).send_keys('\n\n\n'+ second_subtitle).perform()
time.sleep(random.randint(2))
ActionChains(driver).send_keys('\n\n\n'+ third_subtitle).perform()
time.sleep(random.randint(2))
save_button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/div/div[2]/button[1]')
click(save_button)